# Normality

The normality assumption in linear regression refers to the assumption that the target variable is normally distributed for the model of form:

$$ y = \beta_0 + \beta_1 x_1 + \dots + \beta_n x_n $$

As seen in [section 3](3_Linear_regression_MLE.ipynb), this when passed through maximum likelihood estimation (MLE) for $\hat\beta$, we get sum of squared residuals as the cost function which needs to be minimized:

$$ \mathcal{S}(\beta) = \frac{1}{n}\sum_{i=1}^n(y-\hat{y})^2 = \text{MSE} $$

But what-if our target variable is not normally distributed? Benefit is that we can **change the assumption on target variable distribution** and continue with the MLE to get a estimate for $\beta$.

## Breaking the assumption

Lets say, that the data we have in that the target variable instead of being a continuous variable is discrete variable which takes only two values either 0 or 1 (i.e $y \in [0,1]$). This is very common in classification problems where the target variable a class label to which that data point belongs. In this case, the target variable will have binomial distribution:

$$ y \sim Bernoulli(p) $$

which means:

\begin{align*}
    y =
    \left\{
    \begin{array}{ll}
      1,&  p \\
      0,&  1-p
    \end{array}
    \right.
\end{align*}

Let us create a linear model for this estimating the probability of target variable taking a value of 1. This can we defined as

$$
p = \beta_0 + \beta_1 x_1 + \dots + \beta_n x_n
$$

But there is an issue, LHS is a probability which has a range [0,1] whereas RHS has range [$-\infty, +\infty$]. This is a miss match.

What-if instead of modeling probability we model odds ratio i.e. $\frac{p}{1-p}$. This is now unbound on the upper end but still has a lower bound of zero. This ratio has a value of 1 in the middle, indicating a probability of .5 for both occurrence and non occurrence. A small range of odds, from 0 to 1, have a higher probability of failure than for success. Then there is an infinite range of odds, from 1 to infinity, which shows higher probability of success than of failure.

Due to the unbalanced ranges and to centralize the odds ratio around 0, we need to take a logarithmic transformation of the odds ratio. This helps the ranges of the odds ratio to become symmetric around 0, i.e., go from -infinity to +infinity.

So, now we have a model:

$$
\log\Big(\frac{p}{1-p}\Big) = \beta_0 + \beta_1 x_1 + \dots + \beta_n x_n = \beta x
$$

From this we can find a model for probability of y=1

\begin{align*}
\log\Big(\frac{p}{1-p}\Big) &= \beta x & \\
\frac{p}{1-p} &= e^{\beta x} & \text{taking exponential on both side} \\
p &= (1-p)e^{\beta x} & \\
p &= e^{\beta x} - pe^{\beta x} & \\
p + pe^{\beta x} &= e^{\beta x} & \\
p(1 + e^{\beta x}) &= e^{\beta x} & \\
p &= \frac{e^{\beta x}}{(1 + e^{\beta x})} & \\
p &= \frac{1}{(\frac{1}{e^{\beta x}} + \frac{e^{\beta x}}{e^{\beta x}})} & \\
p &= \frac{1}{(\frac{1}{e^{\beta x}} + 1)} & \\
p &= \frac{1}{(e^{-\beta x} + 1)} & \\
p &= \frac{1}{(1 + e^{-\beta x})} & \\
\end{align*}

That is we get probability of y=1:
$$
P(y=1) = \frac{1}{(1 + e^{-\beta x})}
$$

Lets say we have n observed points in our data and if we assume that **each point is independent and identically distributed (iid)**, we can write the likelihood function with respect to all of our observed points as the product of each individual probability density.

For the data point which have target variable value of 1

$$ \mathcal{L}_x(p) = \prod_{y=1} p $$

For the data point which have target variable value of 0

$$ \mathcal{L}_x(p) = \prod_{y=0} (1-p) $$

Combining both of these two we will get

$$ \mathcal{L}_x(p) = \prod^n_{i=1} p^y(1-p)^{(1-y)} $$

The aim to find the parameters of the model that maximize this likelihood.

However, maximizing likelihood function may be complex, therefore, we take a log of both side and convert it to log likelihood

\begin{align*}
\mathcal{l}_x(p) &= \sum^n_{i=1} \log(p^y(1-p)^{(1-y)}) \\
                 &= \sum^n_{i=1} \log(p^y) + \log((1-p)^{(1-y)}) \\
                 &= \sum^n_{i=1} y\log{p} + (1-y)\log{(1-p)}
\end{align*}

Maximizing the log likelihood is same as minimizing the negative log-likelihood. Therefore, the goal becomes:

$$
min \quad -\mathcal{l}_x(p) \\
min \quad -\sum^n_{i=1} y\log{p} + (1-y)\log{(1-p)}
$$

In case of our example, we have already found the value of P(y=1). Let us substitute it in the negative log-likelihood equation:

\begin{align*}
-\mathcal{l}_x(p) &= - \sum^n_{i=1} y_i\log\Big(\frac{1}{(1 + e^{-\beta x_i})}\Big) + (1-y_i)\log{\Big(1 - \frac{1}{(1 + e^{-\beta x_i})}\Big)} \\
                  &= - \sum^n_{i=1} y_i\log 1 - y_i\log{(1 + e^{-\beta x_i})} + (1-y_i)\log{\Big(\frac{1 + e^{-\beta x_i} - 1}{(1 + e^{-\beta x_i})}\Big)} \\
                  &= - \sum^n_{i=1} \log 1^{y_i} - y_i\log{(1 + e^{-\beta x_i})} + (1-y_i)\log{\Big(\frac{e^{-\beta x_i}}{(1 + e^{-\beta x_i})}\Big)} \\
                  &= - \sum^n_{i=1} \log 1 - y_i\log{(1 + e^{-\beta x_i})} + (1-y_i)\log(e^{-\beta x_i}) - (1-y_i)\log{(1 + e^{-\beta x_i})} \\
                  &= \sum^n_{i=1} \log 1 - y_i\log{(1 + e^{-\beta x_i})} - (1-y_i)\beta x_i - (1-y_i)\log{(1 + e^{-\beta x_i})} \\
                  &= - \sum^n_{i=1} \log 1 - y_i\log{(1 + e^{-\beta x_i})} - \beta x_i + y_i\beta x_i - \log{(1 + e^{-\beta x_i})} + y_i\log{(1 + e^{-\beta x_i})} \\
                  &= - \sum^n_{i=1} \log 1 - \beta x_i + y_i\beta x_i - \log{(1 + e^{-\beta x_i})} \\
                  &= - \sum^n_{i=1} \log 1 - \log(e^{\beta x_i}) + y_i\beta x_i - \log{(1 + e^{-\beta x_i})} \\
                  &= - \sum^n_{i=1} \log 1 + y_i\beta x_i - [\log(e^{\beta x_i}) + \log{(1 + e^{-\beta x_i})}] \\
                  &= - \sum^n_{i=1} \log 1 + y_i\beta x_i - \log(e^{\beta x_i}(1 + e^{-\beta x_i})) \\
                  &= - \sum^n_{i=1} \log 1 + y_i\beta x_i - \log(e^{\beta x_i} + e^{\beta x_i}e^{-\beta x_i}) \\
                  &= - \sum^n_{i=1} \log 1 + y_i\beta x_i - \log(1 + e^{\beta x_i}) \\
                  &= - \sum^n_{i=1} y_i\beta x_i + [\log 1 - \log(e^{\beta x_i} + 1)] \\
                  &= - \sum^n_{i=1} y_i\beta x_i + \log\Big(\frac{1}{1 + e^{\beta x_i}}\Big) \\
                  &= \sum^n_{i=1} -y_i\beta x_i - \log\Big(\frac{1}{1 + e^{\beta x_i}}\Big) \\
                  &= \sum^n_{i=1} -y_i\beta x_i + \log\Big(\frac{1}{1 + e^{\beta x_i}}\Big)^{-1} \\
                  &= \sum^n_{i=1} \log(1 + e^{\beta x_i}) - y_i\beta x_i
\end{align*}

Hence, for the example we have chosen to predict the probability of target variable taking value 1, the. estimate of $\beta$ can be found by minimizing the negative log-likelihood

$$
\underset{\beta}{\text{min}} \quad \mathcal{l}_x(\beta) = \sum^n_{i=1} \log(1 + e^{\beta x_i}) - y_i\beta x_i
$$

Since, objective function is non-linear and its derivative is also non-linear, setting the derivative to zero will not give us a closed form solution. Therefore, this need to solved using an iterative solver like:
- Gradient descent
- Newton's method
- Quasi-Newton method

## Segue into Logistic Regression

The change in assumption:

$$ y \sim Bernoulli(p) $$

led to an another popular linear model which is widely used to address binary classification problems. This algorithm is none other than **Logistic Regression**.

### Scikit-learn Implementation

Sklearn provides `LogisticRegression` class to address the logistic regression and it uses **L-BFGS** solver to estimate $\beta$.
Now the question can be asked, why L-BFGS and not any other. Reason are:
- **Gradient descent:**
    - Only uses first-order derivatives (gradients).
    - As the objective function is not quadratic, it does not guarantee convergence in fixed number of step and the convergence can be very slow.
- **Newton’s Method:**
    - Uses second-order derivatives (the Hessian matrix) and can converge quickly in theory, it requires computing and inverting the Hessian matrix at each iteration.
    - For logistic regression, the Hessian is m × m matrix (where m = number of features).
    - For high-dimensional problems (large m), computing and inverting the Hessian is computationally expensive and memory intensive.
    - Full Newton’s Method is not practical for large datasets.
- **Quasi-Newton Methods:**
    - Approximate the Hessian instead of computing it directly.
    - Build an estimate of the inverse Hessian using gradient information from previous iterations.
    - **L-BFGS (Limited-memory BFGS):**
        - Stores only a few vectors to approximate the Hessian
        - Much lower memory usage than full Newton’s method (perfect for high-dimensional problems).
        - Faster convergence than gradient descent.
        - Automatic step size selection (no manual learning rate tuning).
        - It balances speed and computational efficiency.

## Expansion to generalized linear models (GLM)

Like we made the assumption for the target variable to have bernoulli distribution, we have make assumption for target variable to have other distribution from **exponential family** and they give various linear models. Some famous ones are:

- $ y \sim Binomial(n, p) $ -> Binomial regression
- $ y \sim Multinomial(n, k, p) $ -> Multinomial regression
- $ y \sim Poisson(\lambda) $ -> Poisson regression
- $ y \sim Gamma(\alpha, \beta) $ -> Gamma regression